In [1]:
import json
import autogen 
import pandas as pd 
import sqlite3

In [ ]:
config_list = [
    
        {
        "model": "gemini-pro",
        "api_key": "",
        "api_type": "google"
    }
    
]

In [3]:
llm_config = {
    "cache_seed": 47,
    "temperature": 0,
    "config_list": config_list,
    "timeout": 120,
}

In [4]:
#start logging 

import autogen.runtime_logging


logging_session_id = autogen.runtime_logging.start(
    config={"dbname": "logs.db"}
)

In [5]:
print("Started Logging session ID: " + str(logging_session_id))

Started Logging session ID: 8ee8a351-ac19-4251-874b-07e186262bf2


In [6]:
#agent workflow 

assistant = autogen.AssistantAgent(
    name="assistant",llm_config=llm_config
)

user_proxy = autogen.UserProxyAgent(
    name = "user_proxy",
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg= lambda msg : "TERMIATE" in msg["content"]
)

In [8]:
user_proxy.initiate_chat(assistant,message="What is the height of the SEars Towers?Only respond the answer and terminate after one response")

user_proxy (to assistant):

What is the height of the SEars Towers?Only respond the answe

--------------------------------------------------------------------------------
assistant (to user_proxy):

```sh
# filename: get_sears_tower_height.sh
curl -s https://en.wikipedia.org/wiki/Willis_Tower | grep -oP '(?<=Height to roof: )\d+'
```

--------------------------------------------------------------------------------
user_proxy (to assistant):



--------------------------------------------------------------------------------
assistant (to user_proxy):

```sh
# filename: get_sears_tower_height.sh
curl -s https://en.wikipedia.org/wiki/Willis_Tower | grep -oP '(?<=Height to roof: )\d+'
```

--------------------------------------------------------------------------------
user_proxy (to assistant):



--------------------------------------------------------------------------------
assistant (to user_proxy):

```sh
# filename: get_sears_tower_height.sh
curl -s https://en.wikipedia.org/wiki/Wi

KeyboardInterrupt: 

In [9]:
#start logging 

autogen.runtime_logging.stop()

In [10]:
# create function to get log
def get_log(dbname="logs.db", table="chat_completions"):
    con = sqlite3.connect(dbname)
    query = f"SELECT request, response, cost, start_time, end_time from {table}"
    cursor = con.execute(query)
    rows = cursor.fetchall()
    column_names = [description[0] for description in cursor.description]

    data = [dict(zip(column_names, row)) for row in rows]

    con.close()

    return data

In [11]:
def str_to_dict(s):
    return json.loads(s)

In [14]:
log_data = get_log()

In [15]:
log_data_df = pd.DataFrame(log_data)


In [16]:
log_data_df["total_tokens"] = log_data_df.apply(
    lambda row: str_to_dict(row["response"])["usage"]["total_tokens"], axis=1
)

In [17]:
log_data_df["request"] = log_data_df.apply(lambda row: str_to_dict(row["request"])["messages"][0]["content"], axis=1)

In [18]:
log_data_df["response"] = log_data_df.apply(
    lambda row: str_to_dict(row["response"])["choices"][0]["message"]["content"], axis=1
)


In [19]:
print(log_data_df)

                                             request  \
0  You are a helpful AI assistant.\nSolve tasks u...   
1  You are a helpful AI assistant.\nSolve tasks u...   
2  You are a helpful AI assistant.\nSolve tasks u...   
3  You are a helpful AI assistant.\nSolve tasks u...   
4  You are a helpful AI assistant.\nSolve tasks u...   
5  You are a helpful AI assistant.\nSolve tasks u...   
6  You are a helpful AI assistant.\nSolve tasks u...   
7  You are a helpful AI assistant.\nSolve tasks u...   
8  You are a helpful AI assistant.\nSolve tasks u...   

                                            response      cost  \
0  ```sh\n# filename: get_sears_tower_height.sh\n...  0.000323   
1  ```sh\n# filename: get_sears_tower_height.sh\n...  0.000350   
2  ```sh\n# filename: get_sears_tower_height.sh\n...  0.000378   
3  ```sh\n# filename: get_sears_tower_height.sh\n...  0.000405   
4  ```sh\n# filename: get_sears_tower_height.sh\n...  0.000433   
5  ```sh\n# filename: get_sears_tower_heigh